In [ ]:
from google.cloud import storage
import pandas as pd
import pandas as pd
import numpy as np
import requests
import json
from io import StringIO

# Specify the name of your bucket
bucket_name = 'restaurant-dataset'

# Authenticate using a service account key
# Replace 'path/to/your/service-account-key.json' with the path to your service account key file
storage_client = storage.Client.from_service_account_json('/content/nyc-restaurants-419717-6cf4f9e6c135.json')

# Access the specified bucket
bucket = storage_client.bucket(bucket_name)




,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
0,50110282,KARIBBEAN KRAVE,Bronx,65,WESTCHESTER SQ,10461.0,7186736759,Caribbean,08/12/2022,No violations were recorded at the time of thi...,...,Administrative Miscellaneous / Initial Inspection,40.841234,-73.842973,210.0,18.0,20000.0,2097357.0,2.040720e+09,BX37,NaN
1,41450634,YUNG HSIN RESTAURANT,Bronx,147577,BEACH AVENUE,NaN,7184095984,Chinese,05/17/2023,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,0.000000,0.000000,NaN,NaN,NaN,NaN,2.000000e+00,NaN,NaN
2,41459414,NEW YUNG HONG,Bronx,271,WEST KINGSBRIDGE ROAD,10463.0,7185490030,Chinese,06/24/2019,No violations were recorded at the time of thi...,...,Cycle Inspection / Initial Inspection,40.872850,-73.905127,208.0,14.0,27300.0,2015865.0,2.032590e+09,BX28,NaN
3,50150619,GOTHAM STADIUM TENNIS CENTER CAFE,Bronx,725,EXTERIOR STREET,10451.0,9149800750,NaN,01/01/1900,NaN,...,NaN,40.823458,-73.931021,204.0,8.0,6300.0,2097176.0,2.025390e+09,BX63,NaN
4,41232292,PIZZA CHEF,Bronx,564,WEST 235 STREET,10463.0,7187964500,Pizza,04/18/2022,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.885708,-73.910276,208.0,11.0,29700.0,2084102.0,2.057870e+09,BX29,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28355,50119196,SALLY'S SOUTHERN,Staten Island,427,FOREST AVENUE,10301.0,7187272559,Barbecue,08/07/2023,Violations were cited in the following area(s).,...,Pre-permit (Operational) / Re-inspection,40.630824,-74.103542,501.0,49.0,6700.0,5003237.0,5.001320e+09,SI35,NaN
28356,41463477,"DUNKIN',' BASKIN ROBBINS",Staten Island,770,FOREST AVENUE,10310.0,7182739641,Donuts,02/07/2024,Violations were cited in the following area(s).,...,Cycle Inspection / Re-inspection,40.629075,-74.114938,501.0,49.0,12100.0,5008059.0,5.003000e+09,SI35,NaN
28357,41303560,BLU ON THE BAY,Staten Island,492,BAY STREET,10304.0,7184201527,Spanish,06/16/2017,Violations were cited in the following area(s).,...,Cycle Inspection / Re-inspection,40.630008,-74.076740,501.0,49.0,2100.0,5106835.0,5.005100e+09,SI37,NaN
28358,41571795,KUM FUNG KITCHEN,Staten Island,838,BAY STREET,10304.0,7187278888,Chinese,03/14/2022,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.621941,-74.072142,501.0,49.0,4000.0,5042341.0,5.028360e+09,SI08,NaN


In [ ]:
# Specify the folder name within the bucket
folder_name = 'df_raw'

#specify name of the CSV file in the bucket
blob = bucket.blob (f'{folder_name}/StatenIslandBronxInspect.csv')

# Download the CSV file
blob.download_to_filename('StatenIslandBronxInspect.csv')

# Read the CSV file into a DataFrame
statenislandbronx_df = pd.read_csv('StatenIslandBronxInspect.csv')

# Display the DataFrame
statenislandbronx_df

In [ ]:
import math

StatenIslandBronx_df_sorted = StatenIslandBronx_df.sort_values(by=['Latitude', 'Longitude'], ascending=False)
StatenIslandBronx_df_sorted.head()


def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # Radius of Earth in kilometers. Use 3956 for miles
    r = 6371

    # Calculate the result
    return c * r

StatenIslandBronx_df_sorted['prev_latitude'] = StatenIslandBronx_df_sorted['Latitude'].shift(1)
StatenIslandBronx_df_sorted['prev_longitude'] = StatenIslandBronx_df_sorted['Longitude'].shift(1)

# Step 3: Apply the Haversine function to calculate distances between consecutive rows
StatenIslandBronx_df_sorted['distance_to_prev'] = StatenIslandBronx_df_sorted.apply(lambda row: haversine(row['prev_latitude'], row['prev_longitude'], row['Latitude'], row['Longitude']), axis=1)
StatenIslandBronx_df_sorted

# Example usage
lat1, lon1 = 40.648313, -73.788281
lat2, lon2 = 40.685945, -73.793657
distance = haversine(lat1, lon1, lat2, lon2)
print(f"Distance: {distance} kilometers")

Distance: 4.2089819246733775 kilometers


In [ ]:
# Initialize an empty DataFrame to accumulate Yelp data
df_yelp = pd.DataFrame()

#specify name of the CSV file in the bucket
blob = bucket.blob (f'{folder_name}/yelp_data.csv')

# Download the CSV file
blob.download_to_filename('yelp_data.csv')

# Read the CSV file into a DataFrame
processed_restaurants_df = pd.read_csv('yelp_data.csv')

# Display the DataFrame
print(processed_restaurants_df.shape)




(8972, 17)


In [ ]:
# To continue extacting from where left off the last day
# ADJUST THE CSV NAME !!!

# Initialize an empty DataFrame to accumulate Yelp data
df_yelp = pd.DataFrame()

# Specify the name of the file to store processed restaurants' information
processed_restaurants_file = 'yelp_data.csv'

# Check if the file exists in Google Cloud Storage
processed_blob = bucket.blob(processed_restaurants_file)
if processed_blob.exists():
    # Download the file
    processed_blob.download_to_filename(processed_restaurants_file)
    # Read the file into a DataFrame
    processed_restaurants_df = pd.read_csv(processed_restaurants_file)
else:
    # If the file doesn't exist, create an empty DataFrame
    #processed_restaurants_df = pd.DataFrame(columns=['Latitude', 'Longitude'])
    print("does not exist")

# Loop through the DataFrame and print latitude and longitude. Upload Staten Island and Bronx restaurants to GCS
for index, row in StatenIslandBronx_df.iterrows():
    try:
        # Check if the restaurant has already been processed
        if ((processed_restaurants_df['Latitude'] == row['Latitude']) & (processed_restaurants_df['Longitude'] == row['Longitude'])).any():
            print(f"Restaurant at Latitude: {row['Latitude']} and Longitude: {row['Longitude']} already processed.")
            continue  # Skip processing this restaurant

        print(row['DBA'])
        # Make the request to the Yelp API
        url = f"https://api.yelp.com/v3/businesses/search?location=NYC&latitude={row['Latitude']}&longitude={row['Longitude']}&term=restaurants&radius=100&sort_by=review_count&limit=50&offset=1"
        response = requests.get(url, headers=headers)

        # Check the status code
        if response.status_code == 200:
            # Parse the response
            yelp_data = response.json()

            # Check if 'businesses' key has rows
            if 'businesses' in yelp_data and yelp_data['businesses']:
                print("Business data found:")
                # Append Yelp data to the DataFrame
                df_yelp = df_yelp.append(pd.DataFrame(yelp_data['businesses']), ignore_index=True)
            else:
                print("No business data found in the response.")
        else:
            print(f"Request failed with status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        # Handle any exceptions that arise from the request
        print(f"An error occurred: {e}")

# Update processed_restaurants_df with the processed restaurants' information
processed_restaurants_df = processed_restaurants_df.append(StatenIslandBronx_df[['Latitude', 'Longitude']], ignore_index=True)

# Drop duplicates in case the same restaurant is processed multiple times
processed_restaurants_df.drop_duplicates(inplace=True)

# Upload processed_restaurants_df to Google Cloud Storage
processed_blob.upload_from_string(processed_restaurants_df.to_csv(index=False), content_type='text/csv')

# Define blob name and upload CSV to GCS
blob_name = "yelp/raw/all_restaurants.csv"  # Adjust blob name as needed
blob = bucket.blob(blob_name)
output = df_yelp.to_csv(index=False)
blob.upload_from_string(output, content_type='text/csv')

print("All restaurant data from Yelp has been uploaded to Google Cloud Storage.")


In [ ]:
# #THIS ONE PRODUCES INDIVIDUAL CSV
# already used, combined csv files to one "yelp data" in other code

# # Define your Yelp API token
# TOKEN = "um5-Wc71JOtL5RyZ4mLQFRCRIld7DkPZGz3FXdUSHJD0rD4fleKZCxVtexS2gSiZBTabKU8g1h8SCAyTO5iaMAS6FkA8lcv5kve16VAHauLvVmrc0jll8XmYYsQuZnYx"
# headers = {
#     "accept": "application/json",
#     "Authorization": f"Bearer {TOKEN}"
# }

# rows_to_remove = []

# # Loop through the DataFrame and print latitude and longitude. Upload Staten Island and Bronx restaurants to GCS
# for index, row in StatenIslandBronx_df.iterrows():
#     try:
#         print(row['DBA'])
#         # Make the request to the Yelp API. The Radius is 100 meters because I believe based on the coordinates given by NYC, I will find at least the information from
#         url = f"https://api.yelp.com/v3/businesses/search?location=NYC&latitude={row['Latitude']}&longitude={row['Longitude']}&term=restaurants&radius=100&sort_by=review_count&limit=50&offset=1"
#         response = requests.get(url, headers=headers)

#         # Check the status code
#         if response.status_code == 200:
#         # Parse the response
#             yelp_data = response.json()

#             # Check if 'businesses' key has rows
#             if 'businesses' in yelp_data and yelp_data['businesses']:
#                 print("Business data found:")
#                 df_yelp = pd.DataFrame(yelp_data['businesses'])
#                 print(df_yelp)

#                 # Define blob name and upload CSV to GCS
#                 blob_name = f"yelp/raw/{row['DBA']}.csv"  # Adjust blob name as needed
#                 blob = bucket.blob(blob_name)
#                 output = df_yelp.to_csv(index=False)
#                 blob.upload_from_string(output, content_type='text/csv')

#                 print(f"Restaurant: {row['DBA']}, Latitude: {row['Latitude']}, Longitude: {row['Longitude']} has been uploaded to Google Cloud Storage.")
#                 rows_to_remove.append(index)
#             else:
#                 print("No business data found in the response.")
#         else:
#             print(f"Request failed with status code: {response.status_code}")
#     except requests.exceptions.RequestException as e:
#         # Handle any exceptions that arise from the request
#         print(f"An error occurred: {e}")

# # Dropping rows that didn't return anything or where there was an error
# StatenIslandBronx_df.drop(rows_to_remove, inplace=True)
# StatenIslandBronx_df.reset_index(drop=True, inplace=True)

# # Upload the cleaned DataFrame to Google Cloud Storage
# blob_client = bucket.blob(f"/raw/inprocess/restaurants.csv")
# final_output = StatenIslandBronx_df.to_csv(index=False)
# blob_client.upload_from_string(final_output, content_type='text/csv')

Streaming output truncated to the last 5000 lines.
1  {'address1': '987 Prospect Ave', 'address2': '...  +17188424075   
2  {'address1': '921 Prospect Ave', 'address2': '...  +17183281500   
3  {'address1': '963 Prospect Ave', 'address2': '...  +17188600267   
4  {'address1': '910 Prospect Ave', 'address2': '...  +13472709231   
5  {'address1': '948 Prospect Ave', 'address2': '...  +17186170290   
6  {'address1': '963 Prospect Ave', 'address2': N...  +13475910674   
7  {'address1': '923 Prospect Ave', 'address2': N...  +17183288073   

    display_phone    distance  \
0  (718) 942-5862   48.812035   
1  (718) 842-4075   77.021037   
2  (718) 328-1500  103.694741   
3  (718) 860-0267   34.869204   
4  (347) 270-9231   89.955108   
5  (718) 617-0290   42.295767   
6  (347) 591-0674   34.869204   
7  (718) 328-8073   89.966872   

                                          attributes price  
0  {'business_temp_closed': None, 'menu_url': Non...   NaN  
1  {'business_temp_closed': None, 'men

KeyboardInterrupt: 